What is the best way to convert fits to png?

In [ ]:
from astropy.io import fits
import astropy
import numpy
import scipy
import matplotlib.pyplot as plt
import numpy as np

FITS_PATH = '../data/fits/PROMISE-Q1-8micron-filled-v0_3.fits'

SIZE = 800
XSIZE = 800
YSIZE = 800
X = 11000
Y = 4000

# Open currimg and display info using memmap to not load the whole image into memory
imageHDU = fits.open(FITS_PATH, memmap=True, mode='denywrite')
imageHDU.info()

# Get display image as Numpy Array called data
data = np.array(imageHDU[0].data[Y:Y+YSIZE, X:X+XSIZE], dtype=np.float32)

print(data[0][0])

Try to prepare plotting

In [ ]:
import cv2

# When plotting an image to annotate matplotlib will add a border
# around the image. This is not desired so we need to remove it.
# I cannot seem to remove an x border on the left and right of the
# image but i realized that the border is always 25% of the image size
# so we adjust the image size and make it larger on the x axis and
# later adjust the annotated box coordinates to compensate.
ax = plt.axes([0, 0, 1, 1], frameon=False)
ax.get_xaxis().set_visible(False)
ax.get_yaxis().set_visible(False)

plt.autoscale(tight=True)

plt.axis('off')
plt.imshow(data)
plt.savefig('test.jpg')

x_fix, y = int(SIZE/0.75), SIZE
shift_diff_fix = int((x_fix - y) / 2)
  
img = cv2.imread("test.jpg")
img = cv2.resize(img, (x_fix, y))
reset = img.copy()
out = img.copy()

In [ ]:

drawing = False
ix,iy = -1,-1

boxes = []

# define mouse callback function to draw circle
def draw_rectangle(event, x, y, flags, param):
    global ix, iy, drawing, img
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        ix = x
        iy = y
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        new = np.zeros_like(img, np.uint8)
        cv2.rectangle(new, (ix, iy),(x, y),(0, 0, 255),cv2.FILLED)
        out = img.copy()
        alpha = 0.5
        mask = new.astype(bool)
        out[mask] = cv2.addWeighted(img, alpha, new, 1 - alpha, 0)[mask]
        cv2.imshow("Annotation", out)
        img = out.copy()
        boxes.append((ix-shift_diff_fix, iy, x-shift_diff_fix, y))

#Create window
cv2.namedWindow("Annotation")
cv2.setMouseCallback("Annotation", draw_rectangle)

# Keep annotating until escape key is pressed
cv2.imshow("Annotation", out)
while True:
    if cv2.waitKey(33) == ord('s'): break # Save and exit

    if cv2.waitKey(33) == 27: # Exit without saving
        boxes = []
        break

    if cv2.waitKey(33) == ord('r'): # Reset
        boxes = []
        img = reset.copy()
        cv2.imshow("Annotation", out)

    if cv2.waitKey(33) == ord('z'): # Undo
        boxes.pop()
        # TODO FIX SOME WAY TO DISPLAY PREV STATE
        
cv2.destroyAllWindows()

print(boxes)

Try plot stuff

In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans, DBSCAN

distance = 320
core_min = 20

df = pd.read_csv('../data/catalog_v2.1.csv')

sns.set(rc = {'figure.figsize':(20,8)})

dfCoords = df[['x_center', 'y_center', 'approx_size']]
dfCoords['approx_size'] = dfCoords['approx_size'].div(1000).round(2)

clustering = DBSCAN(eps=distance, min_samples=core_min).fit(dfCoords)

sns.set(rc={'axes.facecolor':'black', 'figure.facecolor':'lightgray', 'figure.edgecolor':'darkblue', 'axes.grid':'False'})

sns.relplot(x="x_center", y="y_center", size="approx_size", hue=clustering.labels_,
            alpha=.8, palette="colorblind", height=6, data=dfCoords, sizes=(1, 15), legend = False)


plt.title(f"Distance = {distance} | core_min = {core_min}")
plt.show()

Evaluate accuracy compare two sets of boxes

In [ ]:
import script.cloud_detection as cd
from copy import deepcopy
import script.performance_eval as pe

regfile = '../cnn/regions_to_annotation/box(50316.355, 6033.3671, 234.3168, 178.3296, 0.0).reg'

pe.compare_region_with_catalog(regfile)